In [1]:
import os, random, sys, time, shutil, re, csv, datetime, pickle, math 

import pandas as pd

from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.webdriver import Options

from webdriver_manager.chrome import ChromeDriverManager

from random import seed
from random import random

from bs4 import BeautifulSoup

In [2]:
chrome_options = Options()
#chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument("--log-level=3")  # fatal
chrome_options.add_argument("--start-maximized")

In [3]:
browser = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)

[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324


[WDM] - Driver [/Users/carlos.cayres/.wdm/drivers/chromedriver/mac64/88.0.4324.96/chromedriver] found in cache


In [4]:
browser.get("https://www.linkedin.com/uas/login")

In [5]:
try:
    f = open("../cookies/cookies.pkl")
    f.close()

    print('[Status] Cookies found! Logging in')

    cookies = pickle.load(open("../cookies/cookies.pkl", "rb"))

    for cookie in cookies:
        browser.add_cookie(cookie)
except IOError:
    print('[Status] Cookies not found!')

    print('[Status] Reading username and password file')

    config = open('../config/config.txt')
    lines = config.readlines()
    config.close()
    username = lines[0]
    password = lines[1]

    print('[Status] Entering username and password in their respective fields')

    username_field = browser.find_element_by_id('username')
    username_field.send_keys(username)

    password_field = browser.find_element_by_id('password')
    password_field.send_keys(password)

    print('[Status] Submiting username and password...')

    try:
        password_field.submit()

        print('[Status] Saving cookies...')
        
        pickle.dump(browser.get_cookies(), open("../cookies/cookies.pkl","wb"))
        
        print('[Status] Successfully saved cookies!')
    except:
        pass

[Status] Cookies found! Logging in


In [6]:
#pickle.dump(browser.get_cookies(), open("../cookies/cookies.pkl","wb"))

In [7]:
# browser.current_url
# soup = BeautifulSoup(browser.page_source, features='html.parser')
# company_details = soup.find('dl').find_all()
# company_details_dict = {}
# for num, info in enumerate(company_details):
#     if(info.name == 'dt'):
#         company_details_dict[info.text.replace('\n', '').strip()] = company_details[num+1].text.replace('\n', '').strip()
# company_details_dict
#print(company_details_dict.get('Website', 'test'))
#print(company_details_dict.get('Websites', 'test'))

In [8]:
seed(1)

In [9]:
def get_employees_number(companies):
    
    #opening the csv to save the informations about the client
    try:
        df_info = pd.read_csv('companies_size.csv')
    except:
        df_info = pd.DataFrame()
    
    for processed, company in enumerate(companies):
        print("processed: " + str(processed))
        #searching for the company's linkedin page
        print("searching for the company " + company)
        #print(browser.current_url)
        browser.get("https://www.linkedin.com/search/results/companies/?keywords=" + company)
        time.sleep(2 + random())
        
        #getting company id
        soup = BeautifulSoup(browser.page_source, features='html.parser')
        code = soup.find_all('code')

        company_id = ''

        try:
            for code_str in code:
                if('urn:li:fsd_company:' in code_str.text):
                    company_id = re.findall(r"(?<=fsd_company:)\d+?(?=[,|\"])", code_str.text)[0]
                    #print(company_id)
                    if(company_id != '8175'):
                        break
            if company_id == '' or company_id == '8175':
                raise Exception("No Id for " + company)
        except:

            print("company " + company + " not found\n")
            print(browser.current_url)
            if "results/companies" not in browser.current_url: 
                print('logged out...')
                return
            info_to_append = {
                'company_searched': company, 
                'company_found': '', 
                'found': False, 
                'company_id': '', 
                'employees': '',
                'industry': '',
                'specialties': '',
                'headquarters': ''
                
            }
            df_info = df_info.append(info_to_append, ignore_index=True)
            df_info.to_csv("companies_size.csv", index=False)
            print('---------------------------------------------')
            continue 

        #accessing the company's page
        linkedin_url = "https://www.linkedin.com/company/" + company_id + "/about/"
        browser.get("https://www.linkedin.com/company/" + company_id + "/about/")
        time.sleep(2 + random())
        
        soup = BeautifulSoup(browser.page_source, features='html.parser')
        
        #searching for the company title
        try:
            company_title = soup.find(attrs={"class": "org-top-card-summary__title"})
            company_title = company_title.find("span").getText()
        except:
            try:
                time.sleep(1)
                company_title = soup.find(attrs={"class": "org-top-card-summary__title"})
                company_title = company_title.find("span").getText()
            except:
                company_title = ''
        
        #searching for the number of employees
        try:
            employees = soup.find(attrs={"class": "org-about-company-module__company-size-definition-text"}).getText().replace('\n', '').strip()
            print("employees:", employees)
        except:
            try:
                time.sleep(1)
                soup = BeautifulSoup(browser.page_source, features='html.parser')
                employees = soup.find(attrs={"class": "org-about-company-module__company-size-definition-text"}).getText().replace('\n', '').strip()
            except:
                employees = 'not found'
        
        #searching for the company info:
        company_details_dict = {}
        try:
            company_details = soup.find('dl').find_all()
            for num, info in enumerate(company_details):
                if(info.name == 'dt'):
                    company_details_dict[info.text.replace('\n', '').strip()] = company_details[num+1].text.replace('\n', '').strip()
            print("company details:", company_details_dict)
        except:
            print("Pass")
            pass
        
        
        browser.get('https://www.linkedin.com/jobs/search/?geoId=92000000&f_CR=103644278&f_F=it%2Ceng&f_C=' + company_id)
        time.sleep(1 + random())
        soup = BeautifulSoup(browser.page_source, features='html.parser')
        
        iteng_jobs = ''
        try:
            iteng_jobs = re.search(r"(.+\ )", soup.find("div", attrs={"class": "jobs-search-results-list__title-heading"}).find("small").text).group(1).strip()
        except:
            iteng_jobs = 'not found'
        
        
        #storing company data found
        print('-----------DADOS-----------')
        print('company_searched', company)
        print('company_found', company_title) 
        print('company_id', company_id)
        print('linkedin', linkedin_url)
        print('iteng_jobs', iteng_jobs)
        print('website', company_details_dict.get('Site', ''))
        print('employees', company_details_dict.get('Tamanho da empresa', ''))
        print('industry', company_details_dict.get('Setor', ''))
        print('specialties', company_details_dict.get('Especializações', ''))
        print('headquarters', company_details_dict.get('Sede', ''))
        
        info_to_append = {
            'company_searched': company,
            'company_found': company_title, 
            'found': True, 
            'company_id': company_id,
            'linkedin': linkedin_url,
            'it_eng_jobs': iteng_jobs,
            'website': company_details_dict.get('Site', ''),
            'employees': company_details_dict.get('Tamanho da empresa', ''),
            'industry': company_details_dict.get('Setor', ''),
            'specialties': company_details_dict.get('Especializações', ''),
            'headquarters': company_details_dict.get('Sede', '')
        }
        
        df_info = df_info.append(info_to_append, ignore_index=True)
        df_info.to_csv("companies_size.csv", index=False)
        print('---------------------------------------------')
  

In [15]:
def get_company_searched():
    try:
        searched = pd.read_csv('companies_size.csv')['company_searched'].to_list()
        return list(filter(lambda elm: isinstance(elm, str), searched))
    except:
        return []
len(get_company_searched())

751

In [13]:
def get_company_list():
    try:
        companies = pd.read_csv('Companies.csv')['Company'].to_list()
        return list(filter(lambda elm: isinstance(elm, str), companies))
    except:
        return []
len(get_company_list())

2636

In [14]:
companies = get_company_list()
companies_searched = get_company_searched()
to_search = [c for c in companies if c not in companies_searched]

while len(to_search) > 100:
    
    companies = get_company_list()
    companies_searched = get_company_searched()
    to_search = [c for c in companies if c not in companies_searched]
    
    count = int(20 + 50*random())
    print(count)
    
    print("Buscando total de:", len(to_search))
    get_employees_number(to_search[0:count])
    
    timeSleep = 30 + 60*random()
    print("Esperando um total de:", timeSleep)
    
    time.sleep(timeSleep)

26
Buscando total de: 2293
processed: 0
searching for the company Brash Concepts
employees: 11-50 funcionários
company details: {'Site': 'http://brashconcepts.com/', 'Setor': 'Tecnologia da informação e serviços', 'Tamanho da empresa': '11-50 funcionários', 'Sede': 'Great Neck, New York', 'Tipo': 'Empresa de capital aberto', 'Fundada em': '1980', 'Especializações': 'IT & Network Support, Cloud Services, Security, Business Continuity, Disaster Recovery, Content Filtering, Data Leakage, Anti Virus, Spam, Malware, Real-time location services, asset tracking, panic button, 911 badges, remote workforce e thermal cameras'}
-----------DADOS-----------
company_searched Brash Concepts
company_found Brash Concepts
company_id 3253781
linkedin https://www.linkedin.com/company/3253781/about/
iteng_jobs not found
website http://brashconcepts.com/
employees 11-50 funcionários
industry Tecnologia da informação e serviços
specialties IT & Network Support, Cloud Services, Security, Business Continuity, 

employees: 51-200 funcionários
company details: {'Site': 'http://www.brightsign.biz', 'Setor': 'Hardware', 'Tamanho da empresa': '51-200 funcionários', 'Sede': 'Los Gatos, California', 'Tipo': 'Empresa privada', 'Fundada em': '2002', 'Especializações': 'Solid-state digital signage media players'}
-----------DADOS-----------
company_searched BrightSign
company_found BrightSign LLC
company_id 458455
linkedin https://www.linkedin.com/company/458455/about/
iteng_jobs not found
website http://www.brightsign.biz
employees 51-200 funcionários
industry Hardware
specialties Solid-state digital signage media players
headquarters Los Gatos, California
---------------------------------------------
processed: 12
searching for the company BrightSign LLC
employees: 51-200 funcionários
company details: {'Site': 'http://www.brightsign.biz', 'Setor': 'Hardware', 'Tamanho da empresa': '51-200 funcionários', 'Sede': 'Los Gatos, California', 'Tipo': 'Empresa privada', 'Fundada em': '2002', 'Especializações

company details: {'Site': 'http://freshersrecruit.in', 'Sede': 'HYDERABAD, Telangana'}
-----------DADOS-----------
company_searched Broadridge Financial Solutions
company_found Broadridge Financial Solutions (India) Private Ltd
company_id 38294838
linkedin https://www.linkedin.com/company/38294838/about/
iteng_jobs not found
website http://freshersrecruit.in
employees 
industry 
specialties 
headquarters HYDERABAD, Telangana
---------------------------------------------
processed: 23
searching for the company Broadridge Financial Solutions / ADP Clearing
company Broadridge Financial Solutions / ADP Clearing not found

https://www.linkedin.com/search/results/companies/?keywords=Broadridge%20Financial%20Solutions%20%2F%20ADP%20Clearing
---------------------------------------------
processed: 24
searching for the company Broadridge Financial Solutions, Inc.
company Broadridge Financial Solutions, Inc. not found

https://www.linkedin.com/search/results/companies/?keywords=Broadridge%20Fina

company Bugcrowd, Inc not found

https://www.linkedin.com/search/results/companies/?keywords=Bugcrowd%2C%20Inc
---------------------------------------------
processed: 10
searching for the company Buhler Commercial
employees: 2-10 funcionários
company details: {'Site': 'http://www.buhlercommercial.com', 'Setor': 'Construção', 'Tamanho da empresa': '2-10 funcionários', 'Sede': 'San Francisco, California', 'Tipo': 'Empresa privada', 'Especializações': 'Commercial Construction, Design-Build, Tenant Improvement, Office Renovations, Restaurant Construction, Build Outs e Facility Maintenance'}
-----------DADOS-----------
company_searched Buhler Commercial
company_found Buhler Commercial
company_id 6629111
linkedin https://www.linkedin.com/company/6629111/about/
iteng_jobs not found
website http://www.buhlercommercial.com
employees 2-10 funcionários
industry Construção
specialties Commercial Construction, Design-Build, Tenant Improvement, Office Renovations, Restaurant Construction, Build Out

company Caffeinated Capital Management, Inc. not found

https://www.linkedin.com/search/results/companies/?keywords=Caffeinated%20Capital%20Management%2C%20Inc.
---------------------------------------------
processed: 20
searching for the company Calcula Technologies Inc
company Calcula Technologies Inc not found

https://www.linkedin.com/search/results/companies/?keywords=Calcula%20Technologies%20Inc
---------------------------------------------
processed: 21
searching for the company Calltower Inc.
company Calltower Inc. not found

https://www.linkedin.com/search/results/companies/?keywords=Calltower%20Inc.
---------------------------------------------
processed: 22
searching for the company Calm
employees: 51-200 funcionários
company details: {'Site': 'http://www.calm.com', 'Setor': 'Saúde, bem-estar e educação física', 'Tamanho da empresa': '51-200 funcionários', 'Sede': 'San Francisco, California', 'Tipo': 'Empresa privada', 'Fundada em': '2012', 'Especializações': 'Meditation & R

...

-----------DADOS-----------
company_searched Distribution Management
company_found Distribution Management, Inc
company_id 152746
linkedin https://www.linkedin.com/company/152746/about/
iteng_jobs not found
website http://www.distributionmgmt.com
employees 201-500 funcionários
industry Logística e cadeia de suprimentos
specialties Distribution Excellence, Industry-Leading Technology, Operational Efficiency e Financial Strength and Discipline
headquarters St. Louis, MO
---------------------------------------------
processed: 24
searching for the company DIVERSANT, LLC
employees: 1.001-5.000 funcionários
company details: {'Site': 'http://www.diversant.com', 'Setor': 'Tecnologia da informação e serviços', 'Tamanho da empresa': '1.001-5.000 funcionários', 'Sede': 'Red Bank, New Jersey', 'Tipo': 'Empresa privada', 'Fundada em': '2005', 'Especializações': 'Information Technology Staffing, Contingent Staffing, Project Staffing and Management, Contingent to Full Time, Manage Service Program e 

employees: 51-200 funcionários
company details: {'Site': 'http://www.dogwoodstatebank.com', 'Setor': 'Bancos', 'Tamanho da empresa': '51-200 funcionários', 'Sede': 'Raleigh, North Carolina', 'Tipo': 'Empresa privada'}
-----------DADOS-----------
company_searched Dogwood LLC
company_found Dogwood State Bank
company_id 28515454
linkedin https://www.linkedin.com/company/28515454/about/
iteng_jobs 1
website http://www.dogwoodstatebank.com
employees 51-200 funcionários
industry Bancos
specialties 
headquarters Raleigh, North Carolina
---------------------------------------------
processed: 36
searching for the company Doherty Restoration, Inc.
company Doherty Restoration, Inc. not found

https://www.linkedin.com/search/results/companies/?keywords=Doherty%20Restoration%2C%20Inc.
---------------------------------------------
processed: 37
searching for the company Dolby Laboratories
employees: 1.001-5.000 funcionários
company details: {'Site': 'http://www.dolby.com', 'Setor': 'Produtos eletrô

employees: 11-50 funcionários
company details: {'Tamanho da empresa': '11-50 funcionários', 'Sede': 'San Francisco, California'}
-----------DADOS-----------
company_searched Drawbridge Realty Partners, LP
company_found Drawbridge Realty Partners LP
company_id 63260579
linkedin https://www.linkedin.com/company/63260579/about/
iteng_jobs not found
website 
employees 11-50 funcionários
industry 
specialties 
headquarters San Francisco, California
---------------------------------------------
processed: 5
searching for the company DreamFunded
employees: 11-50 funcionários
company details: {'Site': 'http://www.dreamfunded.com', 'Setor': 'Serviços financeiros', 'Tamanho da empresa': '11-50 funcionários', 'Sede': 'San Francisco, CA', 'Tipo': 'Empresa privada', 'Fundada em': '2014', 'Especializações': 'private investing e equity crowdfunding'}
-----------DADOS-----------
company_searched DreamFunded
company_found DreamFunded 
company_id 3595351
linkedin https://www.linkedin.com/company/3595351

-----------DADOS-----------
company_searched Duggan Corp
company_found Duggan Corp Insurance Services
company_id 13044566
linkedin https://www.linkedin.com/company/13044566/about/
iteng_jobs not found
website https://duggancorp.com
employees 2-10 funcionários
industry Seguros
specialties 
headquarters Burlingame, Californie
---------------------------------------------
processed: 18
searching for the company Dukal Corporation
employees: 51-200 funcionários
company details: {'Site': 'https://www.dukal.com', 'Setor': 'Dispositivos médicos', 'Tamanho da empresa': '51-200 funcionários', 'Sede': 'Ronkonkoma, New York', 'Tipo': 'Empresa privada', 'Fundada em': '1991', 'Especializações': 'Medical Supply Company, Medical Product Manufacturing, Gauze Dressings, Bandages, Cotton & Wood Products, Special Surgicial Dressings, Specialty Dressings, Medical hygiene, advanced wound care, medical tapes, Spa, Veterinary, Medical Supplies, Health Care, Disposable Medical Supplies e Medical Product Distri

KeyboardInterrupt: 

In [16]:
pd.read_csv('companies_size.csv')

,company_found,company_id,company_searched,employees,found,headquarters,industry,it_eng_jobs,linkedin,specialties,website
0,1-800 CONTACTS,15253.0,1-800 CONTACTS,1.001-5.000 funcionários,1.0,"Draper, Utah",Bens de consumo,7,https://www.linkedin.com/company/15253/about/,NaN,http://www.1800contacts.com
1,NaN,NaN,1234 Microtechnologies Inc,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,"140 Proof, Inc.",NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,18 Rabbits Inc.,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4,21e6,67922644.0,21e6 LLC,11-50 funcionários,1.0,"Pfäffikon, Schwyz",Serviços financeiros,not found,https://www.linkedin.com/company/67922644/about/,"Blockchain, Distributed Ledger Technology, Ins...",http://21e6.io/
...,...,...,...,...,...,...,...,...,...,...,...
746,Dukal Corporation,462343.0,Dukal Corporation,51-200 funcionários,1.0,"Ronkonkoma, New York",Dispositivos médicos,not found,https://www.linkedin.com/company/462343/about/,"Medical Supply Company, Medical Product Manufa...",https://www.dukal.com
747,Duke Energy Corporation,4066.0,Duke Energy,+ de 10.001 funcionários,1.0,"Charlotte, NC",Serviços públicos,15,https://www.linkedin.com/company/4066/about/,"power generation, natural gas distribution, po...",http://www.duke-energy.com
748,Duke Energy Corporation,4066.0,Duke Energy Corporation,+ de 10.001 funcionários,1.0,"Charlotte, NC",Serviços públicos,15,https://www.linkedin.com/company/4066/about/,"power generation, natural gas distribution, po...",http://www.duke-energy.com
749,Duratex,42167.0,Duratex,+ de 10.001 funcionários,1.0,"São Paulo, SP",Materiais de construção,not found,https://www.linkedin.com/company/42167/about/,"Painéis de Madeira, Chapas de Fibra, Pisos Lam...",http://www.duratex.com.br


In [17]:
pd.read_csv('companies_size.csv')['found'].value_counts()

1.0    595
0.0    156
Name: found, dtype: int64